In [2]:
kernel.silent(true)

In [1]:
import coursierapi.MavenRepository
interp.repositories() ++= Seq(MavenRepository.of("https://jitpack.io"))

import coursierapi.MavenRepository


In [2]:
import $ivy.`com.github.propi:rdfrules:1.5.0`
import collection._
import org.apache.jena.riot.Lang

import com.github.propi.rdfrules.data._
import com.github.propi.rdfrules.algorithm.amie._
import com.github.propi.rdfrules.algorithm.dbscan._
import com.github.propi.rdfrules.utils._
import com.github.propi.rdfrules.index._
import com.github.propi.rdfrules.rule._
import com.github.propi.rdfrules.ruleset._

import $ivy.$                                

import collection._

import org.apache.jena.riot.Lang


import com.github.propi.rdfrules.data._

import com.github.propi.rdfrules.algorithm.amie._

import com.github.propi.rdfrules.algorithm.dbscan._

import com.github.propi.rdfrules.utils._

import com.github.propi.rdfrules.index._

import com.github.propi.rdfrules.rule._

import com.github.propi.rdfrules.ruleset._

In [3]:
import reflect.io._, Path._
import reflect.io._
import Path._

import reflect.io._, Path._

import reflect.io._

import Path._

In [4]:
val uri = (value: String) => TripleItem.Uri(value)
val qbDataSet = "http://purl.org/linked-data/cube#dataSet";

uri: String => TripleItem.Uri = ammonite.$sess.cmd3$Helper$$Lambda$2803/0x0000000840c6f040@62ff7381
qbDataSet: String = "http://purl.org/linked-data/cube#dataSet"

In [5]:
val graphs = "../data/salaries".toDirectory.files.map(_.path).filter(g => g matches """.*\.ttl""")

graphs: Iterator[String] = non-empty iterator

In [6]:
var salariesCubesMap: Map[String, Graph] = Map()
graphs.foreach(g => {
    val name = g.replaceAll("\\.ttl", "").replaceAll("^.*/", "")
    salariesCubesMap = salariesCubesMap + (name -> Graph(g))
})

salariesCubesMap: Map[String, Graph] = Map(
  "salaries-female" -> com.github.propi.rdfrules.data.Graph@54629b87,
  "salaries-male" -> com.github.propi.rdfrules.data.Graph@542c23b4,
  "salaries-total" -> com.github.propi.rdfrules.data.Graph@17eb2f2
)

In [7]:
import eu.easyminer.discretization.impl.Interval
import eu.easyminer.discretization.impl.IntervalBound._
val rounded = (value: Double, scale: Integer) => BigDecimal(value).setScale(scale, BigDecimal.RoundingMode.HALF_UP).toDouble
val intervalToString = (i: Interval) => "<"+rounded(i.minValue.value,2)+ "__"+rounded(i.maxValue.value,2)+")"

import eu.easyminer.discretization.impl.Interval

import eu.easyminer.discretization.impl.IntervalBound._

rounded: (Double, Integer) => Double = ammonite.$sess.cmd6$Helper$$Lambda$2959/0x0000000840ce6840@280a9e9e
intervalToString: Interval => String = ammonite.$sess.cmd6$Helper$$Lambda$2960/0x0000000840ce7840@7eebaaa6

In [8]:
val averageSalary = TripleItem.Uri("http://data.czso.cz/ontology/prumernaMzda")
val medianSalary = TripleItem.Uri("http://data.czso.cz/ontology/medianMzdy")
val measureUris = Array(averageSalary,medianSalary)

averageSalary: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/prumernaMzda"
)
medianSalary: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/medianMzdy"
)
measureUris: Array[TripleItem.Uri] = Array(
  LongUri("http://data.czso.cz/ontology/prumernaMzda"),
  LongUri("http://data.czso.cz/ontology/medianMzdy")
)

In [9]:
val equiFrequent = (intervalCount: Int) => DiscretizationTask.Equifrequency(intervalCount)

val discretizeEquifrequent = (graph: Graph, intervalsCount: Int, measure: TripleItem.Uri) => {
    graph.filter(t => t.predicate == measure)
         .discretize(equiFrequent(intervalsCount))(quad => quad.triple.predicate == measure)
}

val discretizeEquifrequentAndGetIntervals = (graph: Graph, intervalsCount: Int, measure: TripleItem.Uri) => {
    graph.filter(t => t.predicate == measure)
         .discretizeAndGetIntervals(equiFrequent(intervalsCount))(quad => quad.triple.predicate == measure)
}

equiFrequent: Int => DiscretizationTask.Equifrequency = ammonite.$sess.cmd8$Helper$$Lambda$3007/0x0000000840d05040@5f61b1a8
discretizeEquifrequent: (Graph, Int, TripleItem.Uri) => Graph = ammonite.$sess.cmd8$Helper$$Lambda$3008/0x0000000840d04c40@12851f24
discretizeEquifrequentAndGetIntervals: (Graph, Int, TripleItem.Uri) => IndexedSeq[Interval] = ammonite.$sess.cmd8$Helper$$Lambda$3009/0x0000000840d05840@77fded32

In [10]:
import eu.easyminer.discretization.impl.Interval

val renameInterval = (t: Triple, intervals: IndexedSeq[Interval], suffix: String) => {
    var result = t
    intervals.foreach(interval => {
        val _interval = t.`object`.asInstanceOf[TripleItem.Interval]
        if (_interval.interval == interval) {
            result = t.copy(`object` = TripleItem.Uri(_interval.toString+"_"+suffix+"_"+(intervals.indexOf(interval)+1)+"/"+intervals.size) )
        }
    })
    result
}

val discretize = (graph: Graph, name: String, intervalCounts: Array[Int]) => {
    var result = Dataset() + 
    graph.filter(t => !measureUris.contains(t.predicate))
    .map(t => if (t.predicate.hasSameUriAs(qbDataSet)) t.copy(`object` = uri(name)) else t)
    
        measureUris.foreach(measure => {
            intervalCounts.foreach(intervalCount => {
                val intervals = discretizeEquifrequentAndGetIntervals(graph, intervalCount, measure)
                val measures = discretizeEquifrequent(graph, intervalCount, measure)
                result = result + measures.map(t => {renameInterval(t, intervals, "ef"+intervalCount)})
            })
    })
    result
}

import eu.easyminer.discretization.impl.Interval


renameInterval: (Triple, IndexedSeq[Interval], String) => Triple = ammonite.$sess.cmd9$Helper$$Lambda$3064/0x0000000840d24c40@32f164f6
discretize: (Graph, String, Array[Int]) => Dataset = ammonite.$sess.cmd9$Helper$$Lambda$3065/0x0000000840d38040@39a875c8

In [11]:
val intervalCounts = Array(2, 3, 5, 7)
var dataset = Dataset()
salariesCubesMap
.map(t => discretize(t._2,t._1,intervalCounts))
.foreach(d => dataset = dataset + d)

intervalCounts: Array[Int] = Array(2, 3, 5, 7)
dataset: Dataset = com.github.propi.rdfrules.data.Dataset@6a733dc4

In [12]:
salariesCubesMap
.foreach(d => 
         discretize(d._2,d._1,intervalCounts)
         .cache("../cache/salaries/"+d._1+".cache")
         .export("../exports/salaries/"+d._1+".ttl")
)